In [2]:
import pandas as pd
import constants
import os
from data_cleaning_household import load_combined_raw_data

In [6]:
df_list = []

for i in range(len(url_list)):
    localDestination = "../data/household/"+url_list[i].split("Week")[1]
    zf = ZipFile(localDestination) 
    df = pd.read_csv(zf.open('pulse2020_puf_'+localDestination[18:20]+'.csv'))
    
    df_list.append(df)

In [3]:
base_name='pulse2020_puf_'
df_lists=[]

for f in os.listdir('../data/household/raw'):
    if base_name in f:
        data=pd.read_csv(os.path.join('../data/household/raw',f))
        df_lists.append(data)
# final_data.to_excel(os.path.join('../data',base_name+'all.xlsx'))


In [4]:
df_lists[0]

,SCRAM,WEEK,EST_ST,EST_MSA,PWEIGHT,TBIRTH_YEAR,ABIRTH_YEAR,EGENDER,AGENDER,RHISPANIC,...,COMP1,COMP2,COMP3,INTRNTAVAIL,INTRNT1,INTRNT2,INTRNT3,TSCHLHRS,TTCH_HRS,INCOME
0,V030000001S52011391390122,5,1,NaN,1239.935394,1973,2,1,2,1,...,-88,-88,-88,-88,-88,-88,-88,-88.0,-88.0,3
1,V030000002S02020543300112,5,2,NaN,196.842234,1973,2,2,2,1,...,-99,1,-99,1,-99,1,-99,0.0,0.0,8
2,V030000002S02020880630122,5,2,NaN,295.425365,1951,2,1,2,2,...,-88,-88,-88,-88,-88,-88,-88,-88.0,-88.0,-88
3,V030000002S02020999610122,5,2,NaN,1088.296594,1983,2,1,2,1,...,-99,1,-99,1,-99,1,-99,0.0,0.0,6
4,V030000005S58050092940112,5,5,NaN,20476.738688,1960,2,1,2,1,...,-88,-88,-88,-88,-88,-88,-88,-88.0,-88.0,2
5,V030000005S91051510270112,5,5,NaN,1187.768788,1953,2,2,2,1,...,-88,-88,-88,-88,-88,-88,-88,-88.0,-88.0,2
6,V030000008S19080964780122,5,8,NaN,2383.849099,1944,2,2,2,1,...,-88,-88,-88,-88,-88,-88,-88,-88.0,-88.0,-88
7,V030000008S53080818340112,5,41,NaN,4353.284629,1986,2,2,2,1,...,-88,-88,-88,-88,-88,-88,-88,-88.0,-88.0,7
8,V030000009S30090182130112,5,12,NaN,1088.709353,1949,2,2,2,1,...,-88,-88,-88,-88,-88,-88,-88,-88.0,-88.0,7
9,V030000009S70090246750112,5,9,NaN,2067.386361,1962,2,2,2,1,...,-88,-88,-88,-88,-88,-88,-88,-88.0,-88.0,3


In [ ]:
final_data = pd.concat(df_lists)

In [ ]:
df = pd.read_csv(zf.open('pulse2020_puf_17.csv'))
df = pd.read_csv(zf.open('pulse2020_puf_16.csv'))
df = pd.read_csv(zf.open('pulse2020_puf_15.csv'))
df = pd.read_csv(zf.open('pulse2020_puf_14.csv'))
df = pd.read_csv(zf.open('pulse2020_puf_13.csv'))
df = pd.read_csv(zf.open('pulse2020_puf_12.csv'))

In [8]:
type(df_list[0])

pandas.core.frame.DataFrame

In [ ]:
pulse_data = pd.concat(df_list)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [67]:
url_list[8]

'//www2.census.gov/programs-surveys/demo/datasets/hhp/2020/wk9/HPS_Week09_PUF_CSV.zip'

In [88]:
for i in range(len(url_list)):
    localDestination = "../data/household/"+url_list[i].split("Week")[1]
    print(localDestination[18:20]+'.csv')

17.csv
16.csv
15.csv
14.csv
13.csv
12.csv
11.csv
10.csv
09.csv
08.csv
07.csv
06.csv
05.csv
04.csv
03.csv
02.csv
01.csv


In [ ]:
print(localDestination[18:-4]+'.csv')

In [ ]:
final_data = pd.concat(df_lists, axis=0, sort=False)

In [59]:
weeks = [df_week1, df_week2, df_week3, df_week4, 
         df_week5, df_week6, df_week7, df_week8, 
         df_week9, df_week10, df_week11, df_week12, 
         df_week13]

pulse_df = pd.concat(weeks, axis=0, sort=False)
Preprocessing step 1: Replacement of weeks from integers to dat

'17'